In [659]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,OneHotEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score,GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

In [660]:
df=pd.read_csv('ttrain.csv')
test=pd.read_csv('ttest.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [661]:
df=df.set_index('PassengerId')
test=test.set_index('PassengerId')

In [662]:
df=df.drop(['Name','Ticket','Cabin'],axis='columns')
test=test.drop(['Name','Ticket','Cabin'],axis='columns')

In [663]:
df["Age"].unique()

array([22.  , 38.  , 26.  , 35.  ,   nan, 54.  ,  2.  , 27.  , 14.  ,
        4.  , 58.  , 20.  , 39.  , 55.  , 31.  , 34.  , 15.  , 28.  ,
        8.  , 19.  , 40.  , 66.  , 42.  , 21.  , 18.  ,  3.  ,  7.  ,
       49.  , 29.  , 65.  , 28.5 ,  5.  , 11.  , 45.  , 17.  , 32.  ,
       16.  , 25.  ,  0.83, 30.  , 33.  , 23.  , 24.  , 46.  , 59.  ,
       71.  , 37.  , 47.  , 14.5 , 70.5 , 32.5 , 12.  ,  9.  , 36.5 ,
       51.  , 55.5 , 40.5 , 44.  ,  1.  , 61.  , 56.  , 50.  , 36.  ,
       45.5 , 20.5 , 62.  , 41.  , 52.  , 63.  , 23.5 ,  0.92, 43.  ,
       60.  , 10.  , 64.  , 13.  , 48.  ,  0.75, 53.  , 57.  , 80.  ,
       70.  , 24.5 ,  6.  ,  0.67, 30.5 ,  0.42, 34.5 , 74.  ])

In [664]:
num_pipeline=make_pipeline(SimpleImputer(strategy="median"),StandardScaler())
cat_pipeline=make_pipeline(SimpleImputer(strategy="most_frequent"),OneHotEncoder(handle_unknown="ignore"))

In [665]:
from sklearn.compose import make_column_selector,make_column_transformer

Preprocessing=make_column_transformer((num_pipeline,make_column_selector(dtype_include=np.number)),(cat_pipeline,make_column_selector(dtype_include=object)))

In [666]:
df["Family"]=df["SibSp"]+df["Parch"]+1
df["AgeBucket"]=pd.cut(df["Age"],bins=[0,3,18,35,60,100],labels=['child','Teen','Adult','Senior','old'])

In [667]:
test["Family"]=test["SibSp"]+test["Parch"]+1
test["AgeBucket"]=pd.cut(test["Age"],bins=[0,3,18,35,60,100],labels=['child','Teen','Adult','Senior','old'])

In [668]:
df[["AgeBucket", "Survived"]].groupby(['AgeBucket']).mean()

/var/folders/p0/psf9flvd2wlfhmfwkwqckd740000gn/T/ipykernel_56835/2019219859.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[["AgeBucket", "Survived"]].groupby(['AgeBucket']).mean()


,Survived
AgeBucket,
child,0.666667
Teen,0.458716
Adult,0.382682
Senior,0.400000
old,0.227273


In [669]:
df=df.drop(['Age','Parch','SibSp'],axis='columns')
test=test.drop(['Age','Parch','SibSp'],axis='columns')

In [670]:
X_train=df.drop('Survived',axis='columns')
Y_train=df['Survived']

In [671]:
from imblearn.over_sampling import SMOTE

In [672]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

rf=make_pipeline(Preprocessing,GradientBoostingClassifier(n_estimators=100,random_state=42,max_depth=7,learning_rate=0.01,subsample=0.8))
rf_score=cross_val_score(rf,X_train,Y_train,scoring="accuracy",cv=10)

In [673]:
rf_score.mean()

0.8238077403245943

In [674]:
rf.fit(X_train,Y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x285ed28d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x2828e14c0>)])),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(learning_rate=0.01, max_depth=7,
                                            random_state=42, subsample=0.8))])

In [676]:
rf.score(X_train,Y_train)

0.8799102132435466

In [677]:
pred=rf.predict(test)

In [680]:
output = pd.DataFrame({'PassengerId': test.index, 'Survived': pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
